In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm 
import glob
import argparse
from create_groups import create_hdf5_filter_key

In [2]:
# dataset_path_base="/home/ns/collect_robomimic_demos/sojib_can_expert/PickPlaceCan_01_30_2024_01_45PM_sojib/can_mix_v0.hdf5"
dataset_path_base_lift = "/home/ns/collect_robomimic_demos/Lift_01_30_2024_05_03PM_sojib/lift_mixed_v0.hdf5"

In [3]:
dataset_path_base=dataset_path_base_lift

In [4]:
f_base = h5py.File(dataset_path_base, "r+")
demos = list(f_base["data"].keys())

lengths=[]
demos_minmax={}
for demo_name in demos:
    demo=f_base['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

    action=f_base['data'][demo_name]['actions']
    action=np.array(action) 
    demos_minmax[demo_name] = (np.min(action, axis=0), np.max(action, axis=0))


lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))
print('Median length: ', np.median(lengths))
print('')

for key in f_base['mask'].keys():
    print(key, f_base['mask'][key])

print('')
max_id=np.max([int(demo.split('_')[1]) for demo in demos])
print(f'max_id: {max_id}')

next_id=max_id+1
print(f'next_id: {next_id}')

Number of demos:  300
Max length:  800
Min length:  116
Mean length:  375.99
Median length:  212.0

expert <HDF5 dataset "expert": shape (100,), type "|S16">
failed <HDF5 dataset "failed": shape (100,), type "|S16">
good <HDF5 dataset "good": shape (100,), type "|S16">
train <HDF5 dataset "train": shape (90,), type "|S13">
valid <HDF5 dataset "valid": shape (10,), type "|S12">

max_id: 300
next_id: 301


In [6]:
f_base.close()

In [5]:
# del f_base['mask']['train']
# del f_base['mask']['valid']

In [5]:
def copy_demos(f1, f2, demos2copy, next_id):
    new_demos=[]
    i=next_id
    for demo_name in tqdm.tqdm(demos2copy):
        demo2=f2['data'][demo_name]
        num_samples= demo2.attrs['num_samples']
        model_file= demo2.attrs['model_file']

        bdemo_name="demo_"+str(i)
        new_demos.append(bdemo_name)
        i = i+1

        f1["data"].create_group(bdemo_name)


        ks=['actions', 'dones', 'rewards', 'states']
        for k in ks:
            kdata = demo2[k]
            f1["data"][bdemo_name].create_dataset(k, data=kdata)

        obs_keys =demo2['obs'].keys()
        next_obs_keys= demo['next_obs'].keys()
        for k in obs_keys:
            kdata = demo2['obs'][k]
            f1["data"][bdemo_name].create_dataset(f'obs/{k}', data=kdata)
        for k in next_obs_keys:
            kdata = demo2['next_obs'][k]
            f1["data"][bdemo_name].create_dataset(f'next_obs/{k}', data=kdata)

        f1["data"][bdemo_name].attrs["num_samples"] = num_samples
        f1["data"][bdemo_name].attrs["model_file"] = model_file

    # f.close()
    f2.close()
    return i , new_demos

In [7]:
dataset_paths_can=[
    "/home/ns/collect_robomimic_demos/Alyssa/PickPlaceCan_01_27_2024_04_35PM_AlyssaColandreo/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Benjamin/PickPlaceCan_01_29_2024_02_44PM_Benjamin/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Elbert/PickPlaceCan_01_26_2024_06_40PM_Elbert/demo_image.hdf5", 
    "/home/ns/collect_robomimic_demos/Emma/PickPlaceCan_01_27_2024_10_41AM_Emma/demo_image.hdf5", 
    "/home/ns/collect_robomimic_demos/James/PickPlaceCan_01_27_2024_03_37PM_JaJamesonDunsford/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Lancaster/PickPlaceCan_01_29_2024_01_39PM_lancaster/demo_image.hdf5", 
    "/home/ns/collect_robomimic_demos/Mia/PickPlaceCan_01_28_2024_10_12AM_Mia/demo_image.hdf5", 
    "/home/ns/collect_robomimic_demos/Nalene/PickPlaceCan_01_26_2024_05_17PM_nalene/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Nusrat/PickPlaceCan_01_29_2024_12_03PM_Nusrat/demo_image.hdf5", 
    "/home/ns/collect_robomimic_demos/sumaya/PickPlaceCan_01_27_2024_01_30PM_sumayaakter/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/tazul/PickPlaceCan_01_27_2024_12_40PM_tazulislam/demo_image.hdf5"
]

In [6]:
dataset_paths_lift=[
    "/home/ns/collect_robomimic_demos/Alyssa/Lift_01_27_2024_04_29PM_AlyssaColandreo/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Benjamin/Lift_01_29_2024_02_36PM_Benjamin/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Elbert/Lift_01_26_2024_06_34PM_Elbert/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Emma/Lift_01_27_2024_10_34AM_Emma/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/James/Lift_01_27_2024_03_31PM_JamesonDunsford/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Lancaster/Lift_01_29_2024_01_28PM_Lancaster/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Mia/Lift_01_28_2024_10_00AM_Mia/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Nalene/Lift_01_26_2024_05_12PM_nalene/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/Nusrat/Lift_01_29_2024_11_53AM_Nusrat/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/ola/Lift_01_25_2024_05_25PM_ola/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/sumaya/Lift_01_27_2024_01_20PM_sumayaakter/demo_image.hdf5",
    "/home/ns/collect_robomimic_demos/tazul/Lift_01_27_2024_12_28PM_tazul/demo_image.hdf5"
]

In [7]:
group="try_failed" 
# group="hl"
# group="slow_timeout"

In [8]:
dataset_paths=dataset_paths_lift

In [9]:
new_demoss=[]
for dataset_path in dataset_paths:
    path=os.path.dirname(dataset_path)
    video_group=path+"/videos/"+group
    files=glob.glob(video_group+"/*.mp4")
    demos2copy = [os.path.basename(file).replace(".mp4", "") for file in files]
    

    print(video_group, len(demos2copy))
    
    f2 = h5py.File(dataset_path, "r")
    next_id, new_demos=copy_demos(f_base, f2, demos2copy, next_id)
    new_demoss.append(new_demos)
    f2.close()
    
f_base.close()

/home/ns/collect_robomimic_demos/Alyssa/Lift_01_27_2024_04_29PM_AlyssaColandreo/videos/try_failed 0


0it [00:00, ?it/s]

/home/ns/collect_robomimic_demos/Benjamin/Lift_01_29_2024_02_36PM_Benjamin/videos/try_failed 2


100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


/home/ns/collect_robomimic_demos/Elbert/Lift_01_26_2024_06_34PM_Elbert/videos/try_failed 0


...

100%|██████████| 9/9 [00:08<00:00,  1.12it/s]


/home/ns/collect_robomimic_demos/sumaya/Lift_01_27_2024_01_20PM_sumayaakter/videos/try_failed 22


100%|██████████| 22/22 [00:47<00:00,  2.17s/it]


/home/ns/collect_robomimic_demos/tazul/Lift_01_27_2024_12_28PM_tazul/videos/try_failed 25


100%|██████████| 25/25 [00:21<00:00,  1.16it/s]


In [10]:
new_demoss

[[],
 ['demo_201', 'demo_202'],
 [],
 ['demo_203', 'demo_204'],
 [],
 ['demo_205',
  'demo_206',
  'demo_207',
  'demo_208',
  'demo_209',
  'demo_210',
  'demo_211'],
 ['demo_212',
  'demo_213',
  'demo_214',
  'demo_215',
  'demo_216',
  'demo_217',
  'demo_218',
  'demo_219',
  'demo_220',
  'demo_221',
  'demo_222',
  'demo_223',
  'demo_224',
  'demo_225',
  'demo_226',
  'demo_227',
  'demo_228',
  'demo_229'],
 [],
 ['demo_230',
  'demo_231',
  'demo_232',
  'demo_233',
  'demo_234',
  'demo_235',
  'demo_236',
  'demo_237',
  'demo_238',
  'demo_239',
  'demo_240',
  'demo_241',
  'demo_242',
  'demo_243',
  'demo_244'],
 ['demo_245',
  'demo_246',
  'demo_247',
  'demo_248',
  'demo_249',
  'demo_250',
  'demo_251',
  'demo_252',
  'demo_253'],
 ['demo_254',
  'demo_255',
  'demo_256',
  'demo_257',
  'demo_258',
  'demo_259',
  'demo_260',
  'demo_261',
  'demo_262',
  'demo_263',
  'demo_264',
  'demo_265',
  'demo_266',
  'demo_267',
  'demo_268',
  'demo_269',
  'demo_270'

In [8]:
f_base.close()

In [13]:
nds=np.hstack(new_demoss)
nds.shape

(100,)

In [17]:
demos_group= np.array(np.hstack(new_demoss), dtype='S16') 
demos_group

array([b'demo_201', b'demo_202', b'demo_203', b'demo_204', b'demo_205',
       b'demo_206', b'demo_207', b'demo_208', b'demo_209', b'demo_210',
       b'demo_211', b'demo_212', b'demo_213', b'demo_214', b'demo_215',
       b'demo_216', b'demo_217', b'demo_218', b'demo_219', b'demo_220',
       b'demo_221', b'demo_222', b'demo_223', b'demo_224', b'demo_225',
       b'demo_226', b'demo_227', b'demo_228', b'demo_229', b'demo_230',
       b'demo_231', b'demo_232', b'demo_233', b'demo_234', b'demo_235',
       b'demo_236', b'demo_237', b'demo_238', b'demo_239', b'demo_240',
       b'demo_241', b'demo_242', b'demo_243', b'demo_244', b'demo_245',
       b'demo_246', b'demo_247', b'demo_248', b'demo_249', b'demo_250',
       b'demo_251', b'demo_252', b'demo_253', b'demo_254', b'demo_255',
       b'demo_256', b'demo_257', b'demo_258', b'demo_259', b'demo_260',
       b'demo_261', b'demo_262', b'demo_263', b'demo_264', b'demo_265',
       b'demo_266', b'demo_267', b'demo_268', b'demo_269', b'dem

In [18]:
hdf5_path=dataset_path_base 

# filter_keys=sorted([elem for elem in demos_expert])
# filter_name="expert"
# filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

filter_keys=sorted([elem for elem in demos_group])
filter_name="failed"
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [19]:
f_base.close()

In [3]:
a='abc'

b=a

a='cdf'

a,b

('cdf', 'abc')